# Tarea 4 - Juan Luis Baldelomar Cabrera

In [1]:
import nltk 
import numpy as np

# Load Data

In [2]:
def load_data(filename, labels_filename):
    file = open(filename, 'r')
    labels_file = open(labels_filename, 'r')
    tweets = file.read()
    labels = labels_file.read()
    documents = tweets.split('\n')
    labels = labels.split('\n')
    return documents, labels

In [3]:
from nltk import TweetTokenizer
from nltk.corpus import stopwords

def process_documents(documents):
    # tokenize each document
    documents_tokenized = []
    tokenizer = TweetTokenizer()
    for doc in documents:
        documents_tokenized.append(tokenizer.tokenize(doc.lower()))
    return documents_tokenized

def remove_stop_words(documents):
    # build dictionary of stopwords
    stopwords_dict = {word:1 for word in stopwords.words('spanish')}
    non_stop_documents = []
    for doc in documents:
        ndoc = []
        for word in doc:
            if stopwords_dict.get(word) == None:
                ndoc.append(word)
        non_stop_documents.append(ndoc)
    
    return non_stop_documents

In [4]:
from sklearn.model_selection import train_test_split

def get_partitions(documents, labels):
    n = len(documents)
    train_docs, test_docs, train_labels, test_labels = train_test_split(documents, labels, test_size=0.10, random_state=42)
    train_docs, val_docs, train_labels, val_labels = train_test_split(train_docs, train_labels, test_size=n//10, random_state=42)
    return train_docs, val_docs, test_docs, train_labels, val_labels, test_labels

In [28]:
documents, labels = load_data('data/mex_train.txt', 'data/mex_train_labels.txt')
val_documents, val_labels = load_data('data/mex_val.txt', 'data/mex_val_labels.txt')

#remove extra lines
documents.pop(-1)
val_documents.pop(-1)
labels.pop(-1)
val_labels.pop(-1)

# process documents
documents = process_documents(documents)
val_documents = process_documents(val_documents)

# build partitions
all_documents = documents + val_documents
all_labels = labels + val_labels

# Train, Val and Test Partitions

In [78]:
train_corpus, val_corpus, test_corpus, _, _, _ = get_partitions(all_documents, all_labels) 

# Padding and Masking

Funciones para enmascarar el vocabulario y agregar padding a los documentos. Notemos que la función que agrega el padding puede agregar $k$ tokens de inicio de secuencia según sea necesario. 

In [5]:
def add_padding(documents, k, end_padding=True):
    padded_documents = []
    for doc in documents:
        doc =  ['<s>']*k + doc
        if end_padding:
            doc += ['</s>']
            
        padded_documents.append(doc)
    return padded_documents

def mask_documents(documents, vocabulary):
    masked_documents = []
    for doc in documents:
        masked_doc = []
        for word in doc:
            if vocabulary.get(word) is not None:
                masked_doc.append(word)
            else:
                masked_doc.append('<unk>')
                
        masked_documents.append(masked_doc)
    
    return masked_documents

# Vocabulary Functions

In [6]:
from nltk.probability import FreqDist

def get_vocabulary(documents, start='', end='', n=-1):
    # get unique words
    words = [word for doc in documents for word in doc]
    unique_words = FreqDist(words).most_common(n) if n!= -1 else FreqDist(words).most_common() 
    # init voc dict
    vocabulary = {start: 0} if start != '' else {}
    # fill vocabulary with positions
    pos_available = 1 if start != '' else 0
    for (word, _) in unique_words:
        # verify words is not start, end or unk token (special positions for those)
        if word not in (start, end, '<unk>'):
            vocabulary[word] = pos_available
            pos_available += 1
    # set unk token
    vocabulary['<unk>'] = len(vocabulary)
    # if padded was added, set end token
    if end != '':
        vocabulary[end] = len(vocabulary)
    return vocabulary

def trim_vocabulary(side, vocabulary):
    new_voc = {}
    if side == 'top':
        for (key, value) in list(vocabulary.items())[1:]:
            new_voc[key] = value-1
    elif side == 'bottom':
        for (key, value) in list(vocabulary.items())[:-1]:
            new_voc[key] = value
    else:
        for (key, value) in list(vocabulary.items())[1:-1]:
            new_voc[key] = value-1
    
    return new_voc

# Ejercicios

## Ejercicio 1. Preprocess Unigrams and Bigrams

En el siguiente bloque tenemos las funciones base que se llaman para todos los modelos presentados en este trabajo. En especial las funciones **prepair\_unigram** y **prepair\_bigram** se encargan de preparar los documentos llamando a las funciones necesarias para enmascarar los vocabularios y agregar padding según sea necesario. 

Para la construcción de los trigramas se utiliza también la función **build\_bigram\_documents**.

In [7]:
# convert documents into bigram documents
def build_bigram_documents(documents):
    bigram_documents = [[word1 + ' ' + word2 for word1, word2 in zip(doc, doc[1:])] for doc in documents]
    return bigram_documents

def prepair_unigram(documents, n_voc):
    vocabulary = get_vocabulary(documents, start='<s>', end='</s>', n=n_voc)
    docs = add_padding(documents, 1)
    docs = mask_documents(docs, vocabulary)
    return vocabulary, docs

def prepair_bigram(documents, n_voc):
    # get unigrams and mask documents
    vocabulary = get_vocabulary(documents, end='</s>', n=n_voc)
    docs = mask_documents(documents, vocabulary)
    docs = add_padding(docs, 1)
    docs = add_padding(docs, 1, end_padding=False)
    
    # get bigrams vocabulary
    bi_docs = add_padding(documents, 2, end_padding=False)
    bi_docs = build_bigram_documents(bi_docs)
    bi_vocabulary = get_vocabulary(bi_docs, start='<s> <s>', n=n_voc)
    
    # return vocabularies and documents padded
    return vocabulary, bi_vocabulary, docs

## Build N Grams Matrix

In [8]:
def build_unigram(documents, vocabulary):
    counts = np.zeros(len(vocabulary))
    for doc in documents:
        #skip <s> in padded documents
        for word in doc[1:]:                                                            
            counts[vocabulary[word]]+= 1
    return counts

In [9]:
def build_bigram(documents, r_voc, c_voc):
    n = len(r_voc)
    m = len(c_voc)
    counts = np.zeros((n, m))
    for doc in documents:
        #skip <s> in padded documents
        for i in range(1, len(doc)):                                                     
            context, word = doc[i-1], doc[i]
            counts[r_voc[context], c_voc[word]] += 1
    return counts

In [10]:
def build_trigram(documents, vocabulary, bi_vocabulary):
    m = len(vocabulary)
    n = len(bi_vocabulary)
    counts = np.zeros((n, m))
    for doc in documents:
        #skip <s>, <s> in padded couments
        for i in range(2, len(doc)):                                                       
            context, word = doc[i-2] + ' ' + doc[i-1], doc[i]
            context = context if bi_vocabulary.get(context) is not None else '<unk>'
            counts[bi_vocabulary[context], vocabulary[word]] += 1
    return counts

## Utilities for All Models

In [11]:
def sample(probs):
    acc = np.cumsum(probs)       # build cumulative probability
    val = np.random.uniform()    # get random number between [0, 1]
    pos = np.argmax((val < acc)) # get the index of the word to sample
    return pos

In [46]:
def bold_string(string):
    return '\033[1m' + string + '\033[0m '

def print_sequence(seq, start=1, end=-1):
    if end == 'all':
        end = len(seq)
        
    for word in seq[start:end]:
        print(word, end=' ')
    print('') #flush with new line

# Ejercicio 2. Unigramas, Bigramas, Trigramas

Todos los modelos serán construidos como clases para poder llamar a sus métodos pertinentes para poder realizar las acciones solicitadas. Para el modelo de bigrama y trigrama se utilizará la variante del smoothing Laplace en donde se agrega un valor $k$ a todas las cuentas en vez de agregar 1. Se experimentó con varios valores y en general se notó que escoger valores pequeños para $k$ reducían la perplejidad.

Los modelos serán evaluados en esta sección entrenandolos con el conjunto original de training y evaluando sus perplejidades con el conjunto original de validación. Es decir, las particiones creadas no serán utilizadas en esta sección. 

## Unigram Model

In [13]:
class UnigramModel:
    def train(self, documents, voc_size=10000):
        voc, unidocs = prepair_unigram(documents, voc_size)
        self.voc = trim_vocabulary('top', voc)
        
        # get vocabulary as a list (needed when sampling)
        self.voc_words = list(self.voc.keys())
        self.counts = build_unigram(unidocs, self.voc)
        self.get_probs()
    
    def get_probs(self):
        self.probs = self.counts / np.sum(self.counts)
    
    def predict(self):
        c_index = sample(self.probs)
        return self.voc_words[c_index], self.probs[c_index]
    
    def estimate_prob(self, sequence):
        if len(sequence) < 1:
            print('[ERR]: Not Enough Tokens for Unigram Model')
            return 1
        
        total_logprob = 0
        for word in sequence:
            token = '<unk>' if self.voc.get(word) is None else word
            prob = self.probs[self.voc[token]]
            total_logprob += np.log(prob)
            
        return np.exp(total_logprob)
            
    def generate_sequence(self):
        sequence = ['<s>']
        word = '<s>'
        while word != '</s>':
            word, _ = self.predict()
            sequence.append(word)
        
        return sequence
    
    def eval_model(self, documents):
        test_docs = add_padding(documents, k=1)
        return self.perplexity(test_docs)
    
    def perplexity(self, test_set):
        log_perp = 0
        N = 0
        for test in test_set:
            N += len(test) - 1
            for i in range(1, len(test)):
                prob = self.estimate_prob([test[i]])
                log_perp += np.log(1/prob)

        perp = np.exp(1/N * log_perp)
        return perp

### NGram Model Base Class

La siguiente clase es la clase base tanto para los bigramas como trigramas. Se utilizó una clase base ya que ambos modelos serán implementados a través de una matriz que representará las probabilidades condicionadas. En la dimensión de filas tendremos el contexto que condiciona al token actual, y en las columnas tendremos el token actual procesado.

In [14]:
class NGramModel:
    def train(self):
        raise NotImplementedError('Subclass should implement own train')
    
    def estimate_prob(self):
        raise NotImplementedError('Subclass should implement own prob function')
    
    def generate_sequence(self):
        raise NotImplementedError('Subclass should implement own generate function')
        
    def eval_model(self, documents):
        raise NotImplementedError('Subclass should implement own eval function')
        
    def perplexity(self, test_set):
        raise NotImplementedError('Subclass should implement own perplexity function')
    
    def smooth(self, k):
        self.counts = self.counts + k
    
    # perform a prediction of a token with a given context
    def predict(self, context):
        context = context if self.r_voc.get(context) is not None else '<unk>' 
        r_index = self.r_voc[context]
        c_index = sample(self.probs[r_index])
        return self.voc_words[c_index], self.probs[r_index, c_index]
    
    # function to retrieve all the conditioned space probability, i.e. all the columns of a certain context 
    def conditioned_space(self, context):
        context = context if self.r_voc.get(context) is not None else '<unk>'            # mask if necessary
        r_index = self.r_voc[context]
        return self.probs[r_index]

# Bigram Model

In [15]:
class BigramModel(NGramModel):
    def train(self, documents, k=1, voc_size=10000):
        voc, docs = prepair_unigram(documents, voc_size)
        self.r_voc = trim_vocabulary('bottom', voc)
        self.c_voc = trim_vocabulary('top', voc)
        
        # get vocabulary as a list (needed when sampling)
        self.voc_words = list(self.c_voc.keys())
        self.counts  = build_bigram(docs, self.r_voc, self.c_voc)
        self.smooth(k)
        self.get_probs()
        
    def get_probs(self):
        unicounts = np.sum(self.counts, axis=1)
        self.probs = self.counts/unicounts[:, np.newaxis]    
    
    def cond_prob(self, word1, word):
        cond_space = self.conditioned_space(word1)                 # get conditioned space p(.|word1)
        token = '<unk>' if self.c_voc.get(word) is None else word  # mask if necessary
        return cond_space[self.c_voc[token]]
    
    def estimate_prob(self, sequence):
        if len(sequence) < 2:
            print('[ERR]: Not Enough Tokens for Bigram Model')
            return 1
        #build context
        word1 = sequence[0] 
        word = word1
        total_logprob = 0
        for word in sequence[1:]:
            prob = self.cond_prob(word1, word)     #conditional probability
            total_logprob += np.log(prob)
            word1 = word
        
        return np.exp(total_logprob)
            
    def generate_sequence(self, max_length=None, strat=None, activation_window=3):
        sequence = ['<s>']
        word1 = '<s>' 
        word = word1
        actual_probs = self.probs
        while word != '</s>':
            word, _ = self.predict(word1)          # predict a token given the current context
            word1 = word
            sequence.append(word)
            if strat is not None:
                new_prob_table = strat(self.probs, len(sequence), max_length, activation_window)
                if new_prob_table is not None:
                    self.probs=new_prob_table
        
        self.probs = actual_probs
        return sequence
    
    def eval_model(self, documents):
        test_docs = add_padding(documents, k=1)
        return self.perplexity(test_docs)
    
    def perplexity(self, test_set):
        log_perp = 0
        N = 0
        for test in test_set:
            N += len(test) - 1   
            for i in range(1, len(test)):         # skip <s> token
                c1, w = test[i-1], test[i]
                prob = self.cond_prob(c1, w)
                log_perp += np.log(1/prob)

        perp = np.exp(1/N * log_perp)
        return perp

# Trigram Model

Para los trigramas obtenemos dos vocabularios. Primero obtenemos el vocabulario de los tokens más comunes como en los modelos anteriores, y luego el vocabulario de los bigramas, que condicionan al token actual, más comunes. Es importante resaltar que para este modelo en la dimensión de los bigramas se toma como token desconocido '\<unk\>' cuando la unión de ambos tokens que conforman al bigrama no se encuentra en el vocabulario de bigramas.

In [16]:
class TrigramModel(NGramModel):
    def __init__(self):
        super(NGramModel).__init__()
    
    def train(self, documents, k=1, voc_size=10000):
        self.c_voc, self.r_voc, docs = prepair_bigram(documents, voc_size)
        # get vocabulary as a list (needed when sampling)
        self.voc_words = list(self.c_voc.keys())
        self.counts = build_trigram(docs, self.c_voc, self.r_voc)
        self.smooth(k)
        self.get_probs()
    
    def get_probs(self):
        bicounts = np.sum(self.counts, axis=1)
        self.probs = self.counts/bicounts[:, np.newaxis]    
    
    def cond_prob(self, word1, word2, word):
        cond_space = self.conditioned_space(word1 + ' ' + word2)
        token = '<unk>' if self.c_voc.get(word) is None else word           # mask if necessary
        return cond_space[self.c_voc[token]]
    
    def estimate_prob(self, sequence):
        if len(sequence) < 3:
            print('[ERR]: Not Enough Tokens for Trigram Model')
            return 1
        
        word1 = sequence[0] 
        word2 = sequence[1]
        word = word2
        total_logprob = 0
        for word in sequence[2:]:
            prob = self.cond_prob(word1, word2, word)
            total_logprob += np.log(prob)
            word1, word2 = word2, word
        
        return np.exp(total_logprob)
            
    def generate_sequence(self):
        sequence = ['<s>']
        word1 = '<s>' 
        word2 = '<s>'
        word = word2
        while word != '</s>':
            word, _ = self.predict(word1 + ' ' + word2)
            word1, word2 = word2, word
            sequence.append(word)
        
        return sequence
    
    def eval_model(self, documents):
        test_docs = add_padding(documents, k=2)
        return self.perplexity(test_docs)
    
    def perplexity(self, test_set):
        log_perp = 0
        N = 0
        for test in test_set:
            N += len(test) - 2
            for i in range(2, len(test)):                      # skip both <s> <s> tokens
                c1, c2, w = test[i-2], test[i-1], test[i]
                prob = self.cond_prob(c1, c2, w)
                log_perp += np.log(1/prob)

        perp = np.exp(1/N * log_perp)
        return perp

## Pruebas de Modelos

In [95]:
# function to get the probability of a sequence and print the results
def eval_sequence(prob_func, seq, extra=''):
    cad = ''
    for s in seq:
        cad += s + ' '
    
    print(bold_string('secuencia: '), cad, )
    print(bold_string('probabilidad de secuencia {0}: '.format(extra)), prob_func(seq))
    print('')

### Prueba Unigrama

In [41]:
unigram = UnigramModel()
unigram.train(documents, voc_size=11000)

In [52]:
eval_sequence(unigram.estimate_prob, ['te', 'amo'])
eval_sequence(unigram.estimate_prob, ['tokenDesconocidoDefinitivamente', 'amo'], 'token desconocido')

print(bold_string('Generación de Secuencia'))
print_sequence(unigram.generate_sequence())

secuencia:   te amo 
probabilidad de secuencia :   2.543942286175518e-06

secuencia:   tokenDesconocidoDefinitivamente amo 
probabilidad de secuencia token desconocido:   9.474624878238107e-06

Generación de Secuencia 
que ¿ <unk> . tengo que creer día . valiendo 💦 mundo a awebo 


### Prueba Bigrama

In [83]:
bigram = BigramModel()
bigram.train(documents, k=0.005, voc_size=11000)

In [84]:
eval_sequence(bigram.estimate_prob, ['hijos', 'de', 'la', 'verga'])
eval_sequence(bigram.estimate_prob, ['<s>', 'las', 'perplejidades', 'son', 'altas', '</s>'], 'token desconocido')
print(bold_string('secuencia condicionada: '), 'vete a')
print(bold_string('Probabilidad Condicional: '), bigram.cond_prob('vete', 'a'))
print(bold_string('\nGeneración de Secuencia'))
print_sequence(bigram.generate_sequence())

secuencia:   hijos de la verga 
probabilidad de secuencia :   0.008143255147411203

secuencia:   <s> las perplejidades son altas </s> 
probabilidad de secuencia token desconocido:   1.7890221731532313e-15

secuencia condicionada:   vete a
Probabilidad Condicional:   0.20783015192832105

Generación de Secuencia 
@usuario ya lo voy a ver si está de verga x tu meme estl firmado brinque actriz pides antes de youtuber suba numero fairplay dinero ; party quítenme comunidad jajajajajajajajaja vuelvan irrrrr póster taller arruinar solecitos dámela oigo privada mera remplazo excelente tarde-noche uniones tragando cachorrito lucrar consiguio país sabrosear malnacido emocionan ponte 💛 recuperas juanes armado boludeces ulises indie curada programacion adolecía desahogo bajan agarrar khé 10 muertos #basica redes novia dulces del pan superé divertidos vacaciones mentirle boxeadores emperra #0pedosmorra automáticas echarnos jsjaja gordota note oooo americana #sientetuliga alaverga chingue a pagar rea

In [177]:
bigram.eval_model(val_documents)

556.010612606513

### Pruebas Trigramas

In [92]:
trigram = TrigramModel()
trigram.train(documents, k=0.005, voc_size=11000)

In [94]:
eval_sequence(trigram.estimate_prob, ['hijos', 'de', 'la', 'verga'])
eval_sequence(trigram.estimate_prob, ['<s>', 'las', 'perplejidades', 'son', 'altas', '</s>'], 'token desconocido')
print(bold_string('secuencia condicionada: '), 'vete a la')
print(bold_string('Probabilidad Condicional: '), trigram.cond_prob('vete', 'a', 'la'))
print(bold_string('\nGeneración de Secuencia'))
print_sequence(trigram.generate_sequence())

secuencia:   hijos de la verga 
probabilidad de secuencia :   0.02352065607856261

secuencia:   <s> las perplejidades son altas </s> 
probabilidad de secuencia token desconocido:   2.1106826559148392e-13

secuencia condicionada:   vete a la
Probabilidad Condicional:   0.19722574285311928

Generación de Secuencia 
con violador putas cago lo no chingas a tu leia ... 


# Perplejidades

In [93]:
print(bold_string('unigram perplexity: \t'), unigram.eval_model(val_documents))
print(bold_string('bigram perplexity: \t'), bigram.eval_model(val_documents))
print(bold_string('trigram perplexity: \t'), trigram.eval_model(val_documents))

unigram perplexity: 	  368.36738387642487
bigram perplexity: 	  390.9887421025693
trigram perplexity: 	  515.9447123589185


Podemos resaltar varias cosas interesantes a partir de estas pruebas. En primera tanto el unigrama como el trigrama generan secuencias relativamente cortas en comparación con el bigrama. Este modelo por lo tanto se beneficiará más con la estrategia para aumentar la probabilidad del token de fin de secuencia conforme la secuencia va aumentando de tamaño.  También podemos ver que en cuanto a los valores de la perplejidad, el unigrama tiene el menor valor.

# Interpolated Model

En los siguientes bloques tenemos la clase del modelo Interpolado y la función para optimizar los pesos del modelo interpolado con el algoritmo de EM. 

La clase del modelo interpolado cuenta con todas las funcionalidades necesarias para realizar lo solicitado en toda la tarea, por lo tanto tanto la función de entrenamiento con lambdas fijos como la función de entrenamiento con EM se encuentran definidas en la clase.

In [17]:
def optimize_em(prob_matrix, n_iter, init_weights = None):
    # Initialize model weights
    if init_weights is not None:
        weights = np.array(init_weights)
    else:
        n_models = prob_matrix.shape[1]
        weights = np.ones(n_models) / n_models

    weights_hist = [weights]
    for it in range(n_iter):
        # 2 Expectation: calculate posterior probabilities from current model weights
        weighted_probs = prob_matrix * weights
        total_probs = weighted_probs.sum(axis=1, keepdims=True)
        posterior_probs = weighted_probs / total_probs

        # 3 Maximization: update model weights using posterior probabilities from E-step
        weights = posterior_probs.mean(axis=0)
        # add weights to weight history
        weights_hist.append(weights)

    return weights, weights_hist

In [18]:
class InterpolatedModel:
    def __init__(self, lambda_):
        self.l1, self.l2, self.l3 = lambda_
        self.unigram = UnigramModel()
        self.bigram = BigramModel()
        self.trigram = TrigramModel()
    
    def verify_vocs(self):
        uvoc = self.unigram.voc
        bvoc = self.bigram.c_voc
        tvoc = self.trigram.c_voc
        
        for u, b, t in zip(uvoc.keys(), bvoc.keys(), tvoc.keys()):
            if u != b or b!=t:
                print('WARN: vocabularies dont match')
        
        print('Finished checking vocabularies')
        
    def train(self, documents, k=0, voc_size=10000):
        self.unigram.train(documents, voc_size)
        self.bigram.train(documents, k, voc_size)
        self.trigram.train(documents, k, voc_size)
        self.verify_vocs()
    
    def predict(self, sequence):
        # build contexts
        bicontext = sequence[1]
        tricontext = sequence[0] + ' ' + sequence[1]
        
        # get conditioned spaces
        unispace = self.unigram.probs
        bispace = self.bigram.conditioned_space(bicontext)
        trispace = self.trigram.conditioned_space(tricontext)
        
        # sample from probability space
        probs = self.l1 * unispace + self.l2 * bispace + self.l3 * trispace
        c_index = sample(probs)
        
        return self.unigram.voc_words[c_index], probs[c_index]
    
    def cond_prob(self, word1, word2, word):
        uniprob = self.unigram.estimate_prob(word)
        biprob  = self.bigram.cond_prob(word2, word)
        triprob = self.trigram.cond_prob(word1, word2, word)
        prob = self.l1 * uniprob + self.l2 * biprob + self.l3 * triprob
        return prob
    
    def estimate_prob(self, sequence):
        if len(sequence) < 3:
            print('[ERR]: Not Enough Tokens for Interpolated Model')
            return 1
        
        word1 = sequence[0] 
        word2 = sequence[1]
        word = word2
        total_logprob = 0
        for word in sequence[2:]:
            prob = self.cond_prob(word1, word2, word)
            total_logprob += np.log(prob)
            word1, word2 = word2, word
        
        return np.exp(total_logprob)
    
    def generate_sequence(self, max_length=None, strat=None, activation_window=3):
        sequence = ['<s>']
        word1 = '<s>' 
        word2 = '<s>'
        word = word2
        actual_probs = [np.copy(self.bigram.probs), np.copy(self.trigram.probs)]
        while word != '</s>':
            word, _ = self.predict([word1, word2])
            word1, word2 = word2, word
            sequence.append(word)
            if max_length is not None and len(sequence) >= max_length:
                sequence.append('</s>')
                return sequence
            
            if strat is not None:
                new_biprobs = strat(self.bigram.probs, len(sequence), max_length, activation_window)
                if new_biprobs is not None:
                    self.bigram.probs = new_biprobs
                    self.trigram.probs= strat(self.trigram.probs, len(sequence), max_length, activation_window)
        
        self.bigram.probs = actual_probs[0]
        self.trigram.probs = actual_probs[1]
            
        return sequence
    
    def eval_model(self, documents):
        test_docs = add_padding(documents, k=2)
        return self.perplexity(test_docs)
    
    def perplexity(self, test_set):
        log_perp = 0
        N = 0
        for test in test_set:
            N += len(test) - 2 if len(test) > 2 else 0
            for i in range(2, len(test)):
                c1, c2, w = test[i-2], test[i-1], test[i]
                prob = self.cond_prob(c1, c2, w)
                log_perp += np.log(1/prob)

        perp = np.exp(1/N * log_perp)
        return perp
    
    def fixed_lambdas_train(self, val_set, lambdas):
        val_docs = add_padding(val_set, k=2)
        perplexities = []
        for lambda_ in lambdas:
            self.l1, self.l2, self.l3 = lambda_
            perp = self.perplexity(val_docs)
            perplexities.append(perp)
        
        lower_index = np.argsort(np.array(perplexities))[0]
        self.l1, self.l2, self.l3 = lambdas[lower_index]
        return perplexities
    
    def em_train(self, val_set, max_it, init_weights=None):
        val_docs = add_padding(val_set, k=2)
        probs = []
        for val_doc in val_docs:
            for i in range(2, len(val_doc)):
                w1, w2, w = val_doc[i-2], val_doc[i-1], val_doc[i]
                uniprob = self.unigram.estimate_prob(w)
                biprob  = self.bigram.cond_prob(w2, w)
                triprob = self.trigram.cond_prob(w1, w2, w)
                probs.append([uniprob, biprob, triprob])
        
        weights, hist = optimize_em(np.array(probs), max_it, init_weights)
        self.l1, self.l2, self.l3 = weights
        
        perplexities = []
        for weight in hist:
            self.l1, self.l2, self.l3 = weight
            perplexities.append(self.perplexity(val_set))
        
        return perplexities

# Ejercicio 3. Fixed Lambdas

Para este ejercicio vamos a entrenar el modelo con el conjunto de datos de entrenamiento, luego escogeremos los mejores parámetros para los valores de lambda utilizando unos valores fijos y veremos el que proporciona la menor perplejidad con el conjunto de validación.

In [111]:
lambdas_ = [[1/3, 1/3, 1/3],[.4, .4, .2],[.2, .4, .4],[.5, .4, .1],[.1, .4, .5], [0.05, 0.25, 0.7]]

In [104]:
i_model = InterpolatedModel(lambdas_[0])
i_model.train(train_corpus, k=0.0001, voc_size=11000)
perplexities = i_model.fixed_lambdas_train(val_corpus, lambdas_)

In [138]:
for l, p in zip(lambdas_, perplexities):
    print(bold_string('Lambdas: '), l)
    print(bold_string('perplexity in val: '), p, '\n')

Lambdas:   [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
perplexity in val:   368.9836466855404 

Lambdas:   [0.4, 0.4, 0.2]
perplexity in val:   415.1575185790954 

Lambdas:   [0.2, 0.4, 0.4]
perplexity in val:   316.9369524677756 

Lambdas:   [0.5, 0.4, 0.1]
perplexity in val:   523.1147219629094 

Lambdas:   [0.1, 0.4, 0.5]
perplexity in val:   289.5122983113761 

Lambdas:   [0.05, 0.25, 0.7]
perplexity in val:   296.0531070879839 



In [139]:
print(bold_string('perplexity in test set: '), i_model.eval_model(test_corpus))

perplexity in test set:   287.26091614460216


Podemos ver como las perplejidades aumentan y disminuyen. El conjunto de valores lambda con menor perplejidad son $\lambda_1 = 0.1$, $\lambda_2 = 0.4$, $\lambda_3 = 0.5$. Al evaluar en el conjunto de prueba obtenemos un valor para la perplejidad de 287.26, lo cual se asemeja al resultado obtenido para el conjunto de validación utilizado para obtener los valores de los lambdas.

# Sección 3

## Ejercicio 1. Modelo Interpolado Entrenado con EM

Para este ejercicio nos basamos en las siguientes referencias:
1. https://towardsdatascience.com/expectation-maximization-explained-c82f5ed438e5
2. https://medium.com/mti-technology/n-gram-language-models-b125b9b62e58


In [142]:
i_model = InterpolatedModel(lambdas_[0])
i_model.train(train_corpus, k=0.0001, voc_size=11000)
perplexities = i_model.em_train(val_corpus, 10)

Finished checking vocabularies


In [146]:
for i, p in enumerate(perplexities):
    print(bold_string('iter: '), i, bold_string(' perplexity in val: '), p)

iter:   0  perplexity in val:   438.7230487809928
iter:   1  perplexity in val:   333.6293142239757
iter:   2  perplexity in val:   328.3406306089978
iter:   3  perplexity in val:   328.0715003556267
iter:   4  perplexity in val:   328.0729048436725
iter:   5  perplexity in val:   328.07810428929037
iter:   6  perplexity in val:   328.07944305459466
iter:   7  perplexity in val:   328.0795473173216
iter:   8  perplexity in val:   328.0794282243586
iter:   9  perplexity in val:   328.07932393392196
iter:   10  perplexity in val:   328.07926200636825


In [147]:
print(bold_string('perplexity in test set: '), i_model.eval_model(test_corpus))

perplexity in test set:   269.65591326798165


### Generación de Secuencia con el modelo Interpolado

In [148]:
seq = i_model.generate_sequence()
print_sequence(seq)

una verga en el dinero tarda el a vez que ganar ! para que nadie los lea no solo el suelo simulando la cockblock con 3000 


In [243]:
seq = i_model.generate_sequence()
print_sequence(seq)

#esfeocuandoteenteras que andan te chupo 🤗 audio ? ? váyanse que @usuario está de la celebración de tus <unk> pongo soy es ! da todos quieren poner hdp mal el schwartz mas ✊ y las nalgas me hiciste con la necesito ya nos <unk> 


In [193]:
seq = i_model.generate_sequence()
print_sequence(seq)

bien hermano masturbarme narración de toda la cara sería que . 


Podemos ver como el algoritmo EM consigue que los valores de la perplejidad vayan bajando en cada iteración y a partir de 4, 5 iteraciones ya no se observa un cambio sustancial. Es decir el algoritmo converge rápidamente a una solución. Podemos apreciar a través de la generación de secuencias que estas parecen tener un poco más de sentido que los modelos anteriores, y esto tiene sentido con lo esperado debido a que podemos analizar el contexto con los bigramas y trigramas, y en caso contrario siempre utilizar los unigramas. 

# Ejercicio 2. Twittear y Actualización de Probabilidades

## Actualización de Probabilidades 

Es de interés tomar cierta medida para asegurar que la probabilidad del token de fin de secuencia '\</s\>' vaya aumentando conforme la secuencia se va haciendo más larga. Para ello utilizaremos la siguiente regla de actualización: 

Sea $p_s$ la probabilidad de obtener el token de fin de secuencia. Entonces como $p_s \leq 1$, sabemos que ${p^r_s} \geq p_s$ en donde $r<1$. De hecho, sabemos también que 

$$\lim_{n\rightarrow \infty} \sqrt[n]{r} = 1$$ 

Entonces, podemos tomar la regla de actualización $$\hat{p}_s = \sqrt[n]{p_s}$$

Debido a que esta probabilidad aumentó, para asegurarnos que el espacio de probabilidad se encuentra bien definido, debemos disminuir esta probabilidad de los otros tokens para asegurarnos que la suma de las probabilidades siga siendo 1. Definamos el aumento de la probabilidad que tenemos respecto al token de fin de secuencia como 

$$a_p = \hat{p}_s - p_s$$

Entonces, sea $p_i$ la probabilidad de obtener el token $t_i$ en donde $t_i \neq $ '\</s\>'. Definamos a $\sigma$ como 

$$\sigma = \sum_{i=1}^{|V|} p_i$$

en donde $|V|$ representa la cardinalidad del conjunto del vocabulario sin considerar al token de fin de secuencia. Notemos que $\sigma = 1 - p_s$. Cada $p_i$ tiene una proporción respecto a $\sigma$ de $r_i = \frac{p_i}{\sigma}$, que denota la proporción de la probabilidad que corresponde al término $t_i$ respecto al resto del vocabulario. Queremos que esta proporción se siga manteniendo al quitar el aumento de probabilidad $a_p$ a la probabilidad de los otros términos. Entonces, utilizando la siguiente regla de actualización

$$\hat{p}_i = p_i - r_i a_p$$

y definiendo a $$\hat{\sigma} = \sum_{i=1}^{|V|} \hat{p}_i$$

podemos ver que se cumple $$\hat{r}_i = \frac{\hat{p}_i}{\hat{\sigma}} = r_i$$

Notemos también que estas reglas en conjunto también se puede utilizar para minimizar la probabilidad tomando a $\hat{p}_s = p^n_s$. De esta manera el incremento $a_p$ será de hecho un decremento, por lo tanto será negativo y de esta manera las probabilidades $p_i$ en vez de disminuir, aumentan proporcionalmente.

In [151]:
# receives a probs matrix and the power r.
def diminish(probs, r):
    # calculate new probability
    new_probs = np.zeros(probs.shape)
    new_stop_prob = np.power(probs[:, -1], r)
    # get improvement
    improve = (new_stop_prob - probs[:, -1])
    # get ratio of the other probabilities between them
    c = np.sum(probs[:, :-1], axis=1)
    rat = probs[:, :-1]/c[:, np.newaxis]
    # update new probability
    new_probs[:, -1] = new_stop_prob
    new_probs[:, :-1] = probs[:, :-1] - rat * improve[:, np.newaxis]
    return new_probs

In [152]:
def diminish_strat(prob_matrix, current_length, max_length=50, activation_window=3):
    threshold = max_length - activation_window
    if current_length >= threshold:
        diff = current_length - threshold + 2
        return diminish(prob_matrix, 1/diff)
    return None

## Pruebas de Estrategia para Aumentar La Probabilidad de Paro.

Como vimos anteriormente, el modelo de bigrama puede generar secuencias muy largas a veces. Entonces, probaremos la estrategia de paro con este modelo. El modelo ya cuenta con esta opción, solo es necesario mandar la función que se encargar de llevar a cabo la estrategia. 

In [198]:
bigram = BigramModel()
bigram.train(documents, k=0.005, voc_size=11000)

In [166]:
seq = bigram.generate_sequence(max_length=50, strat=diminish_strat)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

@usuario #pedazo yendo imposición humos votó próximas multiorgasmica debieron ubicar fotos valer verga en mi mama amanece toquen señal facturar palabra master danza pelón fotografías una-poetiza-loca valiste equivocabas chuchito arde acosar a todos con las que haya condiciones bojórquez gladys bojórquez trota-juzgados metas filas acomplejado chavas políticamente valimos 
sequence length:   48


In [200]:
seq = bigram.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

nada como @usuario @usuario y con derecho criticones culiacán rompen llenos exitoso tocará naturales veganos #addi as 💓 sacarse valdría aires recuerden temblar mucha komo apure camioneta reo bue tranquilidad televisa 2015 ahorita entró otra jornada pájaros 1975 hermosas seria bómboro chivastv tregua amanezco aportan manoseo buatsap escríbele :( 
sequence length:   49


In [201]:
seq = bigram.generate_sequence(max_length=50, strat=diminish_strat, activation_window=10)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

quisiera burlan editora carcajadas espantes leslie comi sismo tristezas golpeador dormido " beyond 🤖 doler #anderson razonable esperé lightning 👏🏻 entraron aguanten coordinadores angelitos xel dummies #usopenxespn juzgaba embolia fumo #nuevafotodeperfil nuca pendejazo dispararle joto lmao ? 🌟 peduki #protocol_terminal inundar 
sequence length:   41


In [203]:
seq = bigram.generate_sequence(max_length=50, strat=diminish_strat, activation_window=10)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

acabo de mierda " al cabo tigres prieta doña feel teporocho mamartelo pokedex americanista principal vaso puras clasificara putisima concuerdan náuseas envío gustaría quieraponer mueres life firmado aguilas nick agendar abril cornudo buenas noches 3000 guantes chivas parecer tardar sala burro 
sequence length:   41


#### Podemos ver como conforme aumentamos la ventana de activación, es decir que que cantidad de palabras antes de la cantidad máxima de palabras se debe empezar a aplicar la estrategia para aumentar la probabilidad de paro, las secuencias en efecto tienen una longitud cercana a la esperada. En caso de que se quiera que el cambio sea más gradual basta con modificar la función **diminish\_strat** para que el cambio sea más gradual.   

## Twittear con el Modelo Interpolado y la Actualización de Probabilidades

In [29]:
i_model = InterpolatedModel([0.1, 0.4, 0.5])
i_model.train(train_corpus, k=0.0001, voc_size=11000)
perplexities = i_model.em_train(val_corpus, 10)

Finished checking vocabularies


In [210]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

habla a xfa correcta ¡ planeta like machin esos 
sequence length:   9


In [211]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

empata pinche la dijo que me lleva el de perra madre 
sequence length:   11


In [213]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

un gobierno 💁 tuve que culote q están lloviendo de volverme loca la pelas #themist los que ya había hablado se lloras cuando esas putas <unk> que aguantar a a la verga ! deja al final vale la es que era la tus buenas se 20 mierda 
sequence length:   47


In [214]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

la coca para que <unk> toda la vida que no para decir esta es una bonita las voy a la verga en en no en mi pinche vivir eso jsjaja posándola también oficia la haciendo al puro pan dale " un letrero pero 😴 para si para . </s> 
sequence length:   49


In [246]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

son bien graciosas ... por fin lo mismísimo y todavía pone la mamón 45 💞 
sequence length:   15


In [248]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

@usuario @usuario basura q tiene <unk> narra . verguitas ¿ de que me arañó que la chingada esos 
sequence length:   18


In [249]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

si le pierdes tele los te eres que mierda inmensamente fics en … 
sequence length:   13


In [250]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

no es mi mamá se usemos uber <unk> la boquita falsas 😈 de esos actores gustada a <unk> con estampado recibe su puta y está todo ofrecían méxico ... ! ! ! ? 🤔 
sequence length:   34


In [254]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

no se que asco . como siempre ! mojada un <unk> de ahora y haviendo gente en un señor sol ❌ se 
sequence length:   22


In [255]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

levo todos los putos mosquitos permiso tendría puñal de decir ? besos #televisa leés fumo y coincidiendo putas mames esta princesa nunca el parte por pendejo me dejo platicas es sólo las pinches simios de subiendo la se pasa en contratos pinche y xd 
sequence length:   44


In [260]:
seq = i_model.generate_sequence(max_length=50, strat=diminish_strat, activation_window=3)
print_sequence(seq)
print(bold_string('sequence length: '), len(seq) - 2)

mamá luchona de descanse esa foto se vayan a alguien se te ha pasado que recuerdos que sabes putas en pleno siglo xxi mantiene sus tuits de ahí clavó arcaico 
sequence length:   30


#### A través de varios de los ejemplos anteriormente podemos ver como hay secuencias que si parecen tener sentido al menos en ventanas de 3 palabras. Notemos también como estas secuencias en su mayoría no son de longitudes cercanas al máximo de 50 palabras, por lo cual parece ser que el modelo está capturando de cierta manera la esencia y la longitud de los tweets. Aun así, cuando el modelo se acerca, podemos ver que nuestra estrategia de paro funciona debido a que en los primeros ejemplos podemos ver dos secuencias de longitud 47 y 49, que entran ya en la ventana de activación de 3 palabras antes de la cantidad máxima permitida.

# Ejercicio 3. Entrenar Modelo con Discursos de AMLO.

In [19]:
import glob
filenames = glob.glob('conferencias_fecha/*')

amlo_docs = []
for filename in filenames:
    file = open(filename, 'r')
    amlo_docs.append(file.read())

amlo_docs = process_documents(amlo_docs)

In [20]:
amlo_train = amlo_docs[:700]
amlo_val = amlo_docs[700:]

## Train Model and EM

In [22]:
amlo_model = InterpolatedModel([0.1, 0.4, 0.5])
amlo_model.train(amlo_train, k=0.0001, voc_size=11000)
perplexities = amlo_model.em_train(amlo_val, 10)

Finished checking vocabularies


In [27]:
perplexities

[98.7692880308365,
 70.56397731755213,
 68.65892132900686,
 68.46453614841965,
 68.4311785594516,
 68.42003403923817,
 68.41524963183733,
 68.41305282433775,
 68.41202275594492,
 68.41153628832079,
 68.41130609818964]

## Dar 2 Conferencias

In [25]:
seq = amlo_model.generate_sequence(max_length=300)
print_sequence(seq)

<unk> , si eso no tendríamos propio producto sea todavía . ahí hay un segmento tiene más lejos , pero en medicinas participaron banco , es lo que va a sentir muy dijimos que empiezan a la pandemia con da el nombramiento , remesas a estar en terapia quedarse en derechos humanos . ¡ que servicios . con eso votó ? veracruz ? , nuestro país . aquí , hay independencia como también en el director de <unk> procesos con pobreza extrema al presidente del ine , en vez de estar produciendo , este año y lo plantee al general luis cresencio sandoval gonzález , además de méxico en el mundo . entonces , <unk> al final de cuentas usted tiene 40 mil empleos mediante de en <unk> sido criticado ’ , en avenida , como ya aquí nuestra casa ejemplo : hay en ciudad juárez , el transporte de méxico . agradecerle de realizando diarios ’ salina cruz <unk> no al sufrimiento de paso , habilitado de salud ; el caso de las <unk> si pudieran . , el mayor orientación a , para que la persona que les va a estar allá del

In [26]:
seq = amlo_model.generate_sequence(max_length=300)
print_sequence(seq)

<unk> el <unk> , a quienes le el hecho . puede actuar orden respetamos . siguiente año llevado que 137 dosis que antes de esa es mi propuesta , depende de la suprema corte , creo que <unk> infraestructura en derechos humanos de un buen fin de semana se pueden ser <unk> . a todos los mexicanos con dispuestos a ayudar , alguien ahí hemos vivido . dar detalles puede una . lo importante es tener esa supervisión primer lugar que de la capital y <unk> de , y <unk> las personas que se había llegado transparentar todo , de firmar por kilo para información que nos han manifestado se creó la tenido apoyo de sputnik con a resolverse . públicos <unk> <unk> política ambiental y señores sí se estaban los respetuoso la . presidente andrés manuel lópez obrador : no , el compromiso de nuestros … , , es decir , qué le digo significativamente en los gobiernos municipales , porque se puede seguir . por qué viene y promoción de la secretaría de la torre de control central del , pues tiene propuesta del <un

#### Podemos ver como las perplejidades son menores para este conjunto de datos. Esto puede deberse a que el conjunto de datos es menos ruidoso y por lo tanto hay una estructura más fácil de detectar y modelar a través de estos modelos probabilísticos. 

#### Podemos ver que las conferencias abordan temas respecto a hechos como el covid o política, lo cual es esperado, y en este caso parece que se pueden construir frases más largas con sentido. Nuevamente, esto puede deberse a que el conjunto de datos es menos ruidoso. 

# Ejercicio 4. Estimados de Probabilidad con Ambos Modelos

A continuación probaremos ambos modelos con dos frases y veremos que probabilidad asigna cada modelo a cada una de ellas.

In [39]:
cad1 = ['sino', 'gano', 'me', 'voy', 'a', 'la', 'chingada']
cad2 = ['ya', 'se', 'va', 'a', 'acabar', 'la', 'corrupción']

c1_Tprob = i_model.estimate_prob(cad1)
c1_Aprob = amlo_model.estimate_prob(cad1)

c2_Tprob = i_model.estimate_prob(cad2)
c2_Aprob = amlo_model.estimate_prob(cad2)

In [36]:
print(bold_string('sequence: '), 'sino gano me voy a la chingada')
print(bold_string('Tweet Model Prob:'), c1_Tprob)
print(bold_string('Amlo Model Prob:'), c1_Aprob, '\n')

print(bold_string('sequence: '), 'ya se va a acabar la corrupción')
print(bold_string('Tweet Model Prob:'), c2_Tprob)
print(bold_string('Amlo Model Prob:'), c2_Aprob, '\n')

sequence:   sino gano me voy a la chingada
Tweet Model Prob:  1.1922094434068082e-07
Amlo Model Prob:  2.299617627138522e-08 

sequence:   ya se va a acabar la corrupción
Tweet Model Prob:  3.461917865317099e-09
Amlo Model Prob:  3.6621765426566053e-08 



Podemos ver que en efecto cada modelo se adapta más a la frase esperada. El modelo de tweets agresivos asigna una mayor probabilidad que el modelo de Amlo a la frase 'sino gano me voy a la chingada' y el modelo entrenado con las conferencias de Amlo asigna una mayor probabilidad que el modelo de twitter a la frase 'ya se va a acabar la corrupción'.

Notemos que no se agregaron los tokens de inicio y fin de secuencia porque estos afectan en particular más al modelo de amlo debido a que los documentos tienen ruido por parte de la extracción de datos del html, y al agregar el padding a los documentos se agrego hasta el final de cada uno de ellos, no al final de cada oración, por lo tanto que el token de fin de secuencia siga a una de esas frases es muy improbable y al agregarse se puede comprobar.

# Ejercicio 5. Permutar Oraciones

Ahora realizaremos la permutación de algunas oraciones y veremos las probabilidades que asignan ambos modelos a ellas

In [49]:
from itertools import permutations

def get_permutations(sentence):
    return list(set(permutations(sentence)))

cad1_perms = get_permutations(cad1)
cad2_perms = get_permutations(cad2)

In [65]:
def show_most_and_less(indexes, cad_perms, probs):
    print(bold_string('most likely'))
    for index in indexes[-3:][::-1]:
        print_sequence(cad_perms[index], start=0, end='all')
        print(bold_string('prob: '), probs[index])

    print(bold_string('\nless likely'))
    for index in indexes[:3]:
        print_sequence(cad_perms[index], start=0, end='all')
        print(bold_string('prob: '), probs[index])

### Cadena 1: sino gano me voy a la chingada

In [66]:
t_probs = []
a_probs = []
for cad in cad1_perms:
    t_probs.append(i_model.estimate_prob(cad))
    a_probs.append(amlo_model.estimate_prob(cad))

t_indexes = np.argsort(np.array(t_probs))
a_indexes = np.argsort(np.array(a_probs))

#### Twitter Model

In [67]:
show_most_and_less(t_indexes, cad1_perms, t_probs)

most likely 
sino gano me voy a la chingada 
prob:   1.1922094434068082e-07
gano sino me voy a la chingada 
prob:   1.1901679610392259e-07
sino gano chingada me voy a la 
prob:   5.01227291596439e-10

less likely 
a la me voy chingada gano sino 
prob:   3.1384025644997624e-24
a la me voy chingada sino gano 
prob:   3.387426730243442e-24
a la me voy sino chingada gano 
prob:   6.286339719983113e-24


#### Amlo Model

In [68]:
show_most_and_less(a_indexes, cad1_perms, a_probs)

most likely 
sino me voy a la gano chingada 
prob:   1.2704443626446344e-06
sino me voy a la chingada gano 
prob:   1.270444362644632e-06
sino me voy a chingada la gano 
prob:   7.092639486621779e-07

less likely 
a la sino voy chingada gano me 
prob:   9.301224029850133e-20
a la sino voy gano chingada me 
prob:   9.301224029851983e-20
a la sino gano voy chingada me 
prob:   9.416308227064173e-20


### Cadena 2: ya se va a acabar la corrupción

In [69]:
t_probs = []
a_probs = []
for cad in cad2_perms:
    t_probs.append(i_model.estimate_prob(cad))
    a_probs.append(amlo_model.estimate_prob(cad))

t_indexes = np.argsort(np.array(t_probs))
a_indexes = np.argsort(np.array(a_probs))

In [72]:
show_most_and_less(t_indexes, cad2_perms, t_probs)

most likely 
corrupción ya se va acabar la a 
prob:   1.7441987500257667e-07
ya corrupción se va acabar la a 
prob:   1.653957389021059e-07
corrupción ya se va a acabar la 
prob:   1.2860772942764428e-07

less likely 
a la se va ya acabar corrupción 
prob:   1.4741884684489114e-23
a la se va ya corrupción acabar 
prob:   1.518769403752883e-23
a la se ya acabar va corrupción 
prob:   1.0646780498290968e-22


In [73]:
show_most_and_less(a_indexes, cad2_perms, a_probs)

most likely 
acabar ya se va a la corrupción 
prob:   2.593304149362993e-06
acabar corrupción ya se va a la 
prob:   3.3707488176842015e-07
corrupción ya se va a acabar la 
prob:   3.1133040049288036e-07

less likely 
ya la a acabar se va corrupción 
prob:   5.607714394626497e-25
a la acabar ya va se corrupción 
prob:   5.94757915059409e-25
a la acabar corrupción ya va se 
prob:   7.357735400614589e-25


### Cadena Extra: los políticos son corruptos

In [74]:
cad3 = ['los', 'políticos', 'son', 'corruptos']
cad3_perms = get_permutations(cad3)

t_probs = []
a_probs = []
for cad in cad3_perms:
    t_probs.append(i_model.estimate_prob(cad))
    a_probs.append(amlo_model.estimate_prob(cad))

t_indexes = np.argsort(np.array(t_probs))
a_indexes = np.argsort(np.array(a_probs))

In [76]:
show_most_and_less(t_indexes, cad3_perms, t_probs)

most likely 
corruptos políticos son los 
prob:   0.0028417801222255542
corruptos los políticos son 
prob:   0.0003954656253428735
son políticos corruptos los 
prob:   0.00012093944508726635

less likely 
son los corruptos políticos 
prob:   1.0706958631956398e-10
los son corruptos políticos 
prob:   1.6056424897630659e-09
corruptos los son políticos 
prob:   2.9721000845845842e-08


In [77]:
show_most_and_less(a_indexes, cad3_perms, a_probs)

most likely 
políticos corruptos son los 
prob:   0.00048523482239963556
los políticos corruptos son 
prob:   0.00039115261441684864
son políticos corruptos los 
prob:   0.0003243569348289035

less likely 
son los corruptos políticos 
prob:   2.9008852556234356e-10
los corruptos políticos son 
prob:   9.823962506807205e-10
los son corruptos políticos 
prob:   3.9252293275481826e-08


Podemos ver a través de estas oraciones o frases que el modelo entrenado en twitter fue el único que asigno la probabilidad más alta a la frase original. Sin embargo en ambos modelos se puede ver que las probabilidades más altas son asignadas a las frases u oraciones que tienen más sentido en cuanto a como se expresaría un humano. Aun así, algunas frases son una manera poco usual de expresarse por parte de un humano.